In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/encoded-vector/encoded_vector.pkl


In [2]:
import pickle as pkl
with open('/kaggle/input/encoded-vector/encoded_vector.pkl','rb') as f:
    data=pkl.load(f)
# df=pkl.load('/kaggle/input/encoded-vector/encoded_vector.pkl','rb')


In [3]:
data=data[data.related!=2]

In [4]:
X=data.vector
Y=data.drop(['vector'],axis=1)

In [5]:
Y.shape

(26055, 36)

In [6]:
category_names=Y.columns

In [7]:
category_names

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object', name=0)

In [8]:
((Y.values==2)==True).sum()

0

In [9]:
print(X)
print(Y)

0        [0.19348189, 0.08318756, 0.08449622, -0.174385...
1        [-0.077035, -0.07764, 0.20917, -0.05199, -0.06...
2        [0.15637666, -0.08546334, -0.06660293, -0.2884...
3        [-0.08194833, 0.017492665, 0.018433584, -0.060...
4        [0.035553224, -0.002671669, 0.24559778, -0.178...
                               ...                        
26243    [-0.12471987, -0.058621537, 0.10427699, -0.133...
26244    [-0.023403589, -0.035789352, 0.00350506, -0.18...
26245    [-0.047739904, -0.07247719, 0.024654714, -0.07...
26246    [6.877631e-05, -0.020845791, 0.10321016, -0.04...
26247    [-0.0677657, -0.003936971, 0.09367299, -0.0460...
Name: vector, Length: 26055, dtype: object
0      related  request  offer  aid_related  medical_help  medical_products  \
0            1        0      0            0             0                 0   
1            1        0      0            1             0                 0   
2            1        0      0            0             0              

# Multi-label Classification Methods
Basically, there are three methods to solve a multi-label classification problem, namely:

1. Problem Transformation
2. Adapted Algorithm
3. Ensemble approaches

## 1. Problem Transformation:
1. Binary Relevance
2. Classifier Chains
3. Label Powerset

### Binary Relevance:
 which basically treats each label as a separate single class classification problem.

In [10]:
import skmultilearn
from skmultilearn.adapt import MLkNN
from skmultilearn.problem_transform import BinaryRelevance,ClassifierChain,LabelPowerset
from sklearn.model_selection import StratifiedShuffleSplit,StratifiedGroupKFold,StratifiedKFold,train_test_split
from sklearn.metrics import accuracy_score,classification_report

In [11]:
X_train,X_test,y_train,y_test=train_test_split(X,Y)

In [12]:
# sss=StratifiedShuffleSplit(n_splits=4,test_size=0.2,random_state=200)


In [13]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.multioutput import MultiOutputClassifier

In [14]:
clf_knn=KNeighborsClassifier()
multi_clf=MultiOutputClassifier(clf_knn)

In [15]:
X_train.shape

(19541,)

In [16]:
X_train_v=(np.vstack(X_train.values))


In [17]:
X_test_v=np.vstack(X_test.values)

In [18]:
multi_clf.fit(X_train_v,y_train)


MultiOutputClassifier(estimator=KNeighborsClassifier())

In [19]:
y_pred=multi_clf.predict(X_test_v)

In [20]:
print(classification_report(y_test,y_pred,target_names=category_names))

                        precision    recall  f1-score   support

               related       0.80      0.95      0.87      4989
               request       0.56      0.45      0.50      1149
                 offer       0.00      0.00      0.00        30
           aid_related       0.52      0.80      0.63      2693
          medical_help       0.17      0.32      0.22       519
      medical_products       0.13      0.31      0.18       295
     search_and_rescue       0.05      0.01      0.02       183
              security       0.00      0.00      0.00       113
              military       0.22      0.13      0.16       214
           child_alone       0.00      0.00      0.00         0
                 water       0.14      0.34      0.20       395
                  food       0.34      0.57      0.43       730
               shelter       0.21      0.40      0.28       584
              clothing       0.21      0.09      0.12       105
                 money       0.18      

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1

In [21]:
accuracy_score(y_test,y_pred)

0.12803193122505374

In [22]:
# classifier=MLkNN(k=20)
# classifier.fit(X_train.values,y_train.values)
# classifier.predict(X_test)

In [23]:
# clf=GaussianNB()
# br=ClassifierChain(clf)
# br.fit(X_train,y_train)
# # br.fit(X_train,y_train)

# print(accuracy_score(y_test,br.predict(X_test)))


# # for train_index, test_index in sss.split(X, Y):
# #      print("TRAIN:", train_index, "TEST:", test_index)
# #      X_train, X_test = X[train_index], X[test_index]
# #      y_train, y_test = Y[train_index], Y[test_index]
# #      br.fit(X_train,y_train)
# #      print(accuracy_score(y_test,br.predict(X_test)))
        
# # ValueError: The least populated class in y has only 1 member, 
# # which is too few. The minimum number of groups
# # for any class cannot be less than 2.     